<h1>Project 8</h1>
Update project 5 to support different types of questions. Questions can be 1) Subjective type with long answers, Objective type with a True/False or Objective type with multiple answer choices.
Support an interface that takes a Question and stores it.<br/>
<br/>
Use inheritance to support different types of questions being stored by the implementation of the interface<br/>
<br/>
<h3>Example:</h3>
<br/>
Q1 - Earth is round<br/>
True<br/>
False<br/>
Q2 - What are is the color of a leaf typically<br/>
Red<br/>
Blue<br/>
Green<br/>
White<br/>
Q3 - Describe the properties of steel<br/>
<h3>Requirements</h3>
Implement using the OOPs concepts<br/>
<h3>Error Handling</h3>
As per project 5



In [12]:
#Learning inheritance and interface concept in python
from abc import ABC, abstractmethod

# Define an interface (abstract base class)
class Animal(ABC):
    
    @abstractmethod
    def make_sound(self):
        pass

    @abstractmethod
    def move(self):
        pass


# Implementing the interface in a subclass
class Dog(Animal):
    
    def make_sound(self):
        return "Bark"
    
    def move(self):
        return "Run"


# Another implementation of the Animal interface
class Fish(Animal):
    
    def make_sound(self):
        return "Blub"
    
    def move(self):
        return "Swim"


# Example usage
def main():
    # Instantiate objects of Dog and Fish
    dog = Dog()
    fish = Fish()

    # Call methods defined in the Animal interface
    print(f"Dog says: {dog.make_sound()} and it can {dog.move()}")
    print(f"Fish says: {fish.make_sound()} and it can {fish.move()}")

if __name__ == "__main__":
    main()


Dog says: Bark and it can Run
Fish says: Blub and it can Swim


In [34]:
from abc import ABC, abstractmethod

class Questions(ABC):
    @abstractmethod
    def print_ques(self):
        pass

class Objective(Questions):
    def __init__(self, question_number, question_text, question_options):
        self.question_number = question_number
        self.question_text = question_text
        self.question_options = question_options
        
    def print_ques(self):
        temp_options = ("\n").join(self.question_options)
        print(f"Q{self.question_number}-{self.question_text}\n{temp_options}\n")

class Subjective(Questions):
    def __init__(self, question_number, question_text):
        self.question_number = question_number
        self.question_text = question_text
        
    def print_ques(self):
        print(f"Q{self.question_number}-{self.question_text}\n")

def main():
    objective_ques1 = Objective("1", "Earth is round", ["True", "False"])
    objective_ques1.print_ques()
    objective_ques2 = Objective("2", "What are is the color of a leaf typically", ["Red", "Blue", "Green", "White"])
    objective_ques2.print_ques()
    subjective_ques = Subjective("3","Describe the properties of steel")
    subjective_ques.print_ques()

if __name__ == "__main__":
    main()

Q1-Earth is round
True
False

Q2-What are is the color of a leaf typically
Red
Blue
Green
White

Q3-Describe the properties of steel



In [54]:
import gdown
import re
import PyPDF2
import os
import mysql.connector

class Questions(ABC):
    @abstractmethod
    def print_ques(self):
        pass

class Objective(Questions):
    def __init__(self, question_number, question_text, question_options):
        self.question_number = question_number
        self.question_text = question_text
        self.question_options = question_options
        
    def print_ques(self):
        temp_options = ("\n").join(self.question_options)
        print(f"Q{self.question_number}-{self.question_text}\n{temp_options}\n")

class Subjective(Questions):
    def __init__(self, question_number, question_text):
        self.question_number = question_number
        self.question_text = question_text
        
    def print_ques(self):
        print(f"Q{self.question_number}-{self.question_text}\n")

# Download file from Google Drive
def download_file(work_folder, actual_download_link_url):
    save_file_path = os.path.join(work_folder, "Chemistry Questions.pdf")
    
    try:
        gdown.download(actual_download_link_url, save_file_path, quiet=False)
        print(f"File downloaded successfully at path :{save_file_path}")
    except Exception as e:
        if not os.path.exists(work_folder):
            os.makedirs(work_folder)
        gdown.download(actual_download_link_url, save_file_path, quiet=True)
        print(f"File downloaded successfully at path :{save_file_path}")

# Connect to MySQL Database
def connect_to_mysql_database():
    try:
        connection = mysql.connector.connect(
            host="localhost", 
            user="priya", 
            password="priya", 
            database="mydatabase"
        )
        if connection.is_connected():
            print("Connection to MySQL DB is successful!")
            return connection
    except Exception as e:
        print("Error while connecting to MySQL DB:", e)
        return None

# Check if DB exists or not
def create_db_if_not_exists(connection):
    try:
        cursor = connection.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS mydatabase;")
    except Exception as e:
        print("Error while creating DB:", e)
        return None

# Create table if it doesn't exist
def create_table_if_not_exists(connection):
    cursor = connection.cursor()
    create_table_query = """
    CREATE TABLE IF NOT EXISTS QUESTIONS (
        id INT AUTO_INCREMENT PRIMARY KEY,
        subject_name VARCHAR(100),
        question_text TEXT,
        answer_options TEXT,
        correct_answer VARCHAR(10),
        chapter_name VARCHAR(100)
    );
    """
    cursor.execute(create_table_query)
    connection.commit()


# Insert questions into the database
def insert_questions_into_db(connection, questions):
    cursor = connection.cursor()
    insert_query = """
    INSERT INTO QUESTIONS (subject_name, question_text, answer_options, correct_answer, chapter_name)
    VALUES (%s, %s, %s, %s, %s);
    """
    
    try:
        for question in questions:
            print(f"Inserting question: {question['question_text']}")
            cursor.execute(insert_query, (
                question['subject_name'],
                question['question_text'],
                question['answer_options'],
                question['correct_answer'],
                question['chapter_name']
            ))
        connection.commit()
        print(f"{len(questions)} questions inserted successfully.")
    except Exception as e:
        print("Error inserting data into database:", e)

def main():
    # Connect to MySQL Database
    connection = connect_to_mysql_database()
    
    if connection:
        create_db_if_not_exists(connection)
        # Create table if it doesn't exist
        create_table_if_not_exists(connection)

    # Extract questions from the PDF
    # 1.Store a PDF file in a folder called “/content” 
    download_link_url = "https://drive.google.com/file/d/1OVFr62PNO5-PcnWwFO1Frp0K_0UX6ENc/view?usp=drive_link"
    google_download_file_id = download_link_url.split("/")[5]
    # Corrected URL format for downloading the file
    actual_download_link_url = f'https://drive.google.com/uc?id={google_download_file_id}'
    print("actual_download_link_url : ",actual_download_link_url)
    work_folder = "./content"
    
    # Uncomment to download the file
    #download_file(work_folder, actual_download_link_url)

    pdf_path = os.path.join(work_folder, "Chemistry Questions.pdf")
    # questions = extract_questions_from_pdf(pdf_path)

    
    # Close the database connection
    print()
    objective_ques1 = Objective("1", "Earth is round", ["True", "False"])
    objective_ques1.print_ques()
    objective_ques2 = Objective("2", "What are is the color of a leaf typically", ["Red", "Blue", "Green", "White"])
    objective_ques2.print_ques()
    subjective_ques = Subjective("3","Describe the properties of steel")
    subjective_ques.print_ques()

    questions = []
    temp_questions1 = {}
    temp_questions1['subject_name'] = "Science"
    temp_questions1['question_text'] = objective_ques1.question_text
    temp_questions1['answer_options'] = "\n".join(objective_ques1.question_options)
    temp_questions1['correct_answer'] = "False"
    temp_questions1['chapter_name'] = "Earth"
    questions.append(temp_questions1)
    temp_questions2 = {}
    temp_questions2['subject_name'] = "Science"
    temp_questions2['question_text'] = objective_ques2.question_text
    temp_questions2['answer_options'] = "\n".join(objective_ques2.question_options)
    temp_questions2['correct_answer'] = "False"
    temp_questions2['chapter_name'] = "Plants"
    questions.append(temp_questions2)
    temp_questions3 = {}
    temp_questions3['subject_name'] = "Science"
    temp_questions3['question_text'] = subjective_ques.question_text
    temp_questions3['answer_options'] = ""
    temp_questions3['correct_answer'] = "XXXX"
    temp_questions3['chapter_name'] = "Materials & Matter"
    questions.append(temp_questions3)
    print(questions)

    if questions:
        # Insert extracted questions into the database
        insert_questions_into_db(connection, questions)
    
    connection.close()

if __name__ == "__main__":
    main()


Connection to MySQL DB is successful!
actual_download_link_url :  https://drive.google.com/uc?id=1OVFr62PNO5-PcnWwFO1Frp0K_0UX6ENc

Q1-Earth is round
True
False

Q2-What are is the color of a leaf typically
Red
Blue
Green
White

Q3-Describe the properties of steel

[{'subject_name': 'Science', 'question_text': 'Earth is round', 'answer_options': 'True\nFalse', 'correct_answer': 'False', 'chapter_name': 'Earth'}, {'subject_name': 'Science', 'question_text': 'What are is the color of a leaf typically', 'answer_options': 'Red\nBlue\nGreen\nWhite', 'correct_answer': 'False', 'chapter_name': 'Plants'}, {'subject_name': 'Science', 'question_text': 'Describe the properties of steel', 'answer_options': '', 'correct_answer': 'XXXX', 'chapter_name': 'Materials & Matter'}]
Inserting question: Earth is round
Inserting question: What are is the color of a leaf typically
Inserting question: Describe the properties of steel
3 questions inserted successfully.


In [ ]:
import mysql.connector

In [66]:
# Connect to MySQL Database
connection = connect_to_mysql_database()

if connection:
    create_db_if_not_exists(connection)
    # Create table if it doesn't exist
    create_table_if_not_exists(connection)

mycursor = connection.cursor()
mycursor.execute("select * from QUESTIONS where chapter_name='Plants';")
myresult = mycursor.fetchall()

myresult

Connection to MySQL DB is successful!


[(51,
  'Science',
  'What are is the color of a leaf typically',
  'Red\nBlue\nGreen\nWhite',
  'False',
  'Plants')]